### Inntak

Inntakið er stefnt net af Höfuðborgarsvæðinu, hver hnútur v táknar mót tveggja vega og
hefur auðkenni og staðsetningu, (x, y) hnit. Leggirnir (u, v) tákna vegi, þ.e. það er hægt að
keyra á frá u til v og á milli þessara hnúta er ekki hægt að taka neinar aðrar ákvarðanir í
umferðinni. Skráin nodes inniheldur upplýsingar um hnútana, edges inniheldur upplýsingar um vegina.

### Reiknirit

Ef við fáum gefinn lista af k hnútum þar sem settar hafa verið upp hleðslustöðvar, búið til
reiknirit sem reiknar stystu fjarlægð frá hnverjum hnúti v í einhverja hleðslustöð. Fjarlægðir
á leggjum eiga að vera mældar í metrum og þær eru reiknaðar út frá hnitum hnútanna.
Einfaldasta leiðin til að leysa þetta er að nota reiknirit dijkstra fyrir hverja hleðslustöð
en það er hægt að hraða því með því að setja upphafsfjarlægðirnar á skynsamlegan hátt.

## Verkþættir

### 1. Þáttun (⋆)

Lesið inn netið úr skránum sem eru gefnar, nodes.tsv og edges.tsv. Í skránni nodes.tsv
eru hnútar með auðkenni (id), hnit (x, y) og hvort þeir séu á aðalvegi (primary). Í skránni
edges.tsv eru leggi frá hnúti u til hnúts v með lengd length, mæld í metrum, og nafn
(name).

In [5]:
import pandas as pd

In [6]:
nodes = pd.read_csv("nodes.tsv", sep="\t")
edges = pd.read_csv("edges.tsv", sep="\t")

In [7]:
import networkx as nx

# fall sem býr til net
def make_graph(nodes, edges):
    """Býr til nx.DiGraph með gefnum nodes og edges"""

    # Búum til stefnt net
    G = nx.DiGraph()

    # Bætum við hnútum
    for _, row in nodes.iterrows():
        G.add_node(row["osmid"], x=row["x"], y=row["y"], primary=row["primary"])

    # Bætum við leggjum
    for _, row in edges.iterrows():
        G.add_edge(row["u"], row["v"], weight=row["length"])
    
    return G 


In [8]:
def create_graph_dict(nodes, edges):
    """
    Creates a graph dictionary from nodes and edges DataFrames.

    Parameters:
    - nodes: DataFrame containing node information.
    - edges: DataFrame containing edge information.

    Returns:
    - graph: Dictionary representing the graph.
    """
    # Initialize an empty dictionary for the graph
    graph = {node_id: {} for node_id in nodes["osmid"]}

    # Populate the graph with edges
    for _, row in edges.iterrows():
        u = row["u"]  # Source node
        v = row["v"]  # Target node
        weight = row["length"]  # Edge weight
        graph[u][v] = weight

    return graph

In [9]:
# Búum til net með gefnum nodes og edges
G = create_graph_dict(nodes, edges)

# create a smaller graph for testing
small_nodes = nodes[nodes["osmid"].isin([252743977, 252744017, 252744019, 252744020, 252744087, 252744169, 252744265, 252744269, 252744311, 252744761, 10799990340, 10799990350, 837627438, 1194972614, 10799990319, 10799990311, 837627420, 837627414, 837627410, 8585855191, 1194973102])]

pos = {row["osmid"]: (row["x"], row["y"]) for _, row in small_nodes.iterrows()}

filtered_edges = edges[
            edges["u"].isin(pos.keys()) & edges["v"].isin(pos.keys())
]
G_small = make_graph(small_nodes, filtered_edges) 

print(G_small)


DiGraph with 21 nodes and 38 edges


In [10]:
G_small_dict = create_graph_dict(small_nodes, filtered_edges)
print(G_small_dict)

{252743977: {10799990340: 11.166247287788131, 10799990350: 31.676114024679695}, 252744017: {252744761: 36.45911409065883, 252744169: 116.91047923722422, 252744019: 61.7120569183292}, 252744019: {252744020: 47.37947782346877, 837627438: 45.570294770686935, 252744017: 61.7120569183292}, 252744020: {252744019: 47.37947782346877}, 252744087: {1194972614: 33.01112627463227, 10799990319: 11.314953109531311}, 252744169: {10799990311: 10.182186591756205, 252744017: 116.91047923722422, 252744265: 196.18408969547883}, 252744265: {252744269: 139.64964990481604, 252744169: 196.1840896954789, 837627420: 88.80142636069803}, 252744269: {837627414: 147.38608464875284, 252744265: 139.64964990481607, 252744311: 69.30303181583486}, 252744311: {837627410: 420.6498541177572, 252744269: 69.30303181583486, 8585855191: 142.57439488797942}, 252744761: {252744017: 36.45911409065883, 1194973102: 351.33322227161443, 837627414: 158.34728019618612}, 837627410: {252744311: 420.6498541177571}, 837627414: {252744269: 

### 2. Leit (⋆⋆)

Ef við setjum hleðslustöðvar á hnúta v1, . . . , vk þá er hægt að nota reikniritið dijkstra til að finna stystu fjarlægð frá hverjum hnúti u í hleðslustöð vi. Útfærið reikniritið sem tekur inn lista af lokahnútum og reiknar fjarlægðir frá öllum hnútum í netinu. 
Athugið að netið er stefnt net.

In [11]:
import heapq

def reverse_graph(graph):
    reversed_graph = {node: {} for node in graph}
    for u in graph:
        for v, w in graph[u].items():
            if v not in reversed_graph:
                reversed_graph[v] = {}
            reversed_graph[v][u] = w
    return reversed_graph

def dijkstra(graph, start):
    queue = [(0, start)]
    distances = {node: float('inf') for node in graph}
    previous = {node: None for node in graph}
    distances[start] = 0

    while queue:
        current_dist, current_node = heapq.heappop(queue)
        if current_dist > distances[current_node]:
            continue

        for neighbor, weight in graph[current_node].items():
            distance = current_dist + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous[neighbor] = current_node
                heapq.heappush(queue, (distance, neighbor))

    return distances, previous

def reconstruct_path(previous, start, end):
    path = []
    current = start
    while current != end and current is not None:
        path.append(current)
        current = previous[current]
    if current == end:
        path.append(end)
        path.reverse()
        return path
    return []  # no path found


def shortest_paths_to_stations(graph, destination_nodes):
    reversed_graph = reverse_graph(graph)
    results = {}  # {node: {dest: (distance, path)}}

    for dest in destination_nodes:
        distances, previous = dijkstra(reversed_graph, dest)

        for node in distances:
            if node not in results:
                results[node] = {}
            dist = distances[node]
            path = reconstruct_path(previous, node, dest)
            results[node][dest] = (dist, path)

    return results

### 3. Framsetning (⋆)

Setjið fimm hleðslustöðvar í netið og sýnið stystu leið fyrir fimm punkta og teiknið upp á kort. Tékkið ykkur af með því að bera saman leiðina sem er fundin og fjarlægðina miðað við kortavefi eins og t.d. Google Maps.

In [12]:
# Veljum fimm hnúta þar sem við setjum hleðslustöð
charging_stations = [12885876, 111456955, 14586813, 26471955, 34187360]

# fimm hnútar sem við veljum
five_nodes = [34389202, 35295068, 253467906, 330050169, 27237169]

results_five_stations = shortest_paths_to_stations(G, charging_stations)


In [13]:
import folium

def plot_paths_on_map(dijkstra_results, nodes, origin_filter=None):

    # Convert nodes DataFrame into lookup: id -> (lat, lon)
    coord_lookup = nodes.set_index("osmid")[["y", "x"]].to_dict("index")  # (lat, lon)

    # Get first available coordinate for map center
    if origin_filter:
        first_origin = next((o for o in origin_filter if o in coord_lookup), None)
    else:
        first_origin = next((o for o in dijkstra_results if o in coord_lookup), None)
    
    if first_origin is None:
        raise ValueError("No valid origin node found in coord_lookup.")

    first_coord = coord_lookup[first_origin]
    m = folium.Map(location=[first_coord["y"], first_coord["x"]], zoom_start=14)

    # Draw filtered paths
    for origin, destinations in dijkstra_results.items():
        if origin_filter and origin not in origin_filter:
            continue

        for dest, (dist, path) in destinations.items():
            if not path or any(node not in coord_lookup for node in path):
                continue  # Skip incomplete paths

            # Get lat/lon path
            latlon_path = [(coord_lookup[n]["y"], coord_lookup[n]["x"]) for n in path]

            # Draw path
            folium.PolyLine(latlon_path, color="blue", weight=3, opacity=0.6).add_to(m)

            # Mark origin
            folium.Marker(
                latlon_path[0],
                icon=folium.Icon(color="green", icon="play"),
                popup=f"Upphafspunktur: {origin}"
            ).add_to(m)

            # Mark destination
            folium.Marker(
                latlon_path[-1],
                icon=folium.Icon(color="blue", icon="charging-station", prefix="fa"),
                popup=f"Hleðslustöð: {dest}<br>Stysta fjarlægð: {dist:.1f} m"
            ).add_to(m)

    return m


In [14]:
map = plot_paths_on_map(results_five_stations, nodes, origin_filter=five_nodes)
map

### 4. Tímamælingar (⋆)

Mælið tímann sem reikniritið dijkstra tekur að reikna allar fjarlægðir í netinu með fimm hleðslustöðvum.

In [16]:
import time

start_time = time.time()

dijkstra_results = shortest_paths_to_stations(G, charging_stations)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Reiknirit Djikstra tók: {elapsed_time:.4f} sekúndur")

Reiknirit Djikstra tók: 0.3045 sekúndur


### 5. A* Reikniritið (⋆⋆)
Útfærið A⋆ reikniritið sem tekur inn lista af lokahnútum og reiknar fjarlægðir frá öllum hnútum í netinu. Sem neðra mat á fjarlægð á milli hnútanna má taka d(u, v) =p(xu − xv )2 + (yu − yv )2, þ.e. beina loftlínu milli punktanna. Mælið tíma og berið saman við reiknirit Dijkstra.

In [17]:
import heapq
import math

# fall sem tekur inn net og lista af hleðslustöðvum i því neti og skilar lista af öllum hnútum og styðstu leið þeirra til hleðslustöðvar
def shortest_paths_to_charging_stations_using_Astar(G, charging_stations):
    """Reiknirit sem finnur styðstu leið alla punkta til hleðslustöðvar"""

    heuristic = {
        node: min(math.sqrt((G.nodes[node]["x"] - G.nodes[cs]["x"])**2 + (G.nodes[node]["y"] - G.nodes[cs]["y"])**2) 
               for cs in charging_stations)
        for node in G.nodes
    }

    results = {} # geymir styðstu leið fyrir hvern punkt

    for start in G.nodes:  # Keyrum A* fyrir hvern einasta punkt
        # smá setup
        open_set = [(0, start)]  # setjum inn upphafspunkt með ekkert weight
        g_score = {n: float('inf') for n in G.nodes} # látum g gildi alla punkta verða óendanleg
        g_score[start] = 0 # upphafspunkturinn tekur engan tíma til að komast í

        # ítrum í gegnum priority queue-ið
        while open_set:
            _, current = heapq.heappop(open_set) # fáum léttasta hnútinn

            if current in charging_stations:  # ef við erum stödd á hleðslustöð
                results[start] = g_score[current] # setjum inn styðstu vegalengdina fyrir upphafspunktinn í results
                break  # þurfum ekki að gera neitt meira því hleðslustöð fannst

            for neighbor in G.neighbors(current): # ítrum í gegnum nágranna punktsins sem við völdum
                weight = G[current][neighbor]["weight"] # finnum lengd milli nágranna punktsins og punktsins sem við völdum
                tentative_g = g_score[current] + weight # lengd frá upphafspunkt í þennan nágranna ef notað er besta leiðin til valda punktsins

                if tentative_g < g_score[neighbor]: # kíkjum hvort að nýja lengdin sé styttri en sú sem nágranninn hefur nú þegar
                    g_score[neighbor] = tentative_g # breytum g gildi nágrannans
                    f_score = tentative_g + heuristic[neighbor] # reiknum f gildi hans
                    heapq.heappush(open_set, (f_score, neighbor)) # setjum nágrannan með f gildinu hans í priority queue-ið okkar

        # gerist eftir að while lykjan er búin
        if start not in results:  # þetta gerist ef hleðslustöð var aldrei fundin
            results[start] = float('inf') # segjum að lengdin sé óendanleg

    return results

In [18]:
import heapq
import math
from collections import defaultdict

def shortest_paths_to_charging_stations_using_Astar_faster(G, charging_stations, nodes):
    """Reiknirit sem finnur styðstu leið allra punkta til hleðslustöðvar"""
    
    # þetta er mjög svipað og A* fyrir ofan nema í staðinn fyrir að fara í gegnum hvern hnút og finna lengdina
    # þá er farið út frá hleðslustöðvunum í alla hnúta og þá er ekki verið að endur reikna neitt
    # ætla ekki að gera comments fyrir þetta strax því þetta er mjög svipað hinu 

    # Build a coordinate lookup from DataFrame
    coords = nodes.set_index('osmid')[['x', 'y']].to_dict('index')

    heuristic = {
        n: min(math.sqrt((coords[n]["x"] - coords[cs]["x"])**2 + (coords[n]["y"] - coords[cs]["y"])**2) 
               for cs in charging_stations)
        for n in G
    }

    open_set = []
    g_score = defaultdict(lambda: float('inf'))

    for cs in charging_stations:
        g_score[cs] = 0
        heapq.heappush(open_set, (heuristic[cs], cs))

    while open_set:
        _, current = heapq.heappop(open_set)

        for neighbor, weight in G.get(current, {}).items():
            tentative_g = g_score[current] + weight

            if tentative_g < g_score[neighbor]:
                g_score[neighbor] = tentative_g
                f_score = tentative_g + heuristic[neighbor]
                heapq.heappush(open_set, (f_score, neighbor))

    return dict(g_score)


In [19]:
# samanburður á A-star og Dijkstra
import time

charging_stations = [12885876, 111456955, 14586813, 26471955, 34187360]

start_time = time.time()

results_astar = shortest_paths_to_charging_stations_using_Astar_faster(G, charging_stations, nodes)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Reiknirit A-star tók: {elapsed_time:.4f} sekúndur")
# Reiknirit A-star tók: 0.1827 sekúndur

start_time = time.time()

results_dijkstra = shortest_paths_to_stations(G, charging_stations)

end_time = time.time()
print(f"Reiknirit Djikstra tók: {elapsed_time:.4f} sekúndur")
# Reiknirit Djikstra tók: 3.5279 sekúndur

Reiknirit A-star tók: 0.0685 sekúndur
Reiknirit Djikstra tók: 0.0685 sekúndur


### 6. Staðsetning hleðslustöðva (⋆⋆)

Ef við setjum k hleðslustöðvar í hnúta v1, . . . , vk þá látum við markfallið vera
F (v1, . . . , vk) = X
u∈V
min
i=1,...,k d(u, vi)
þ.e. fyrir hvern hnút í netinu reiknum við stystu fjarlægð frá honum til næstu hleðslustöðvar
og leggjum saman yfir alla hnúta í netinu. Finnið bestu lausn fyrir k = 1, með því að prófa alla hnúta sem hægt er að setja
hleðslustöð í og veljið þann sem gefur minnsta markfall. Athugið að eingöngu þeir hnútar
sem eru merktir sem primary geta verið hleðslustöðvar.

In [20]:
def average_distance(results):
    """reiknum meðal lengd hnúta frá hleðslustöð"""
    penalty_value = 10**7 # þurfum að taka með að sumir hnútar geta ekki náð í hleðslustöð svo þá er lengdin inf en það eyðileggur average-ið
    distances = [
        dist if dist != float('inf') else penalty_value # svo í staðinn breytum við inf lengdum í stóra tölu
        for dist in results.values() # ( það virkar ekki að taka út allar inf tölur því þá erum við ekki að fá raunverulegt average )
        ]
    if not distances:  # Ef enginn hnútur nær til hleðslustöðvar
        return None # skilum við bara None

    return sum(distances) / len(distances) # samtals lengd / magn hnúta

In [21]:
possible_charging_stations = nodes[nodes["primary"]]["osmid"].tolist()
print(f'Fjöldi mögulegra hleðslustöðva: {len(possible_charging_stations)}')

start_time = time.time()

lowest_dist = float('inf')
best_charger = None
for charging_station in possible_charging_stations:
    results = shortest_paths_to_charging_stations_using_Astar_faster(G, [charging_station], nodes)
    avrg_dist = average_distance(results)
    print(f"Hleðslustöð: {charging_station}, meðalvegalengd (m): {avrg_dist}")
    if avrg_dist is not None and avrg_dist < lowest_dist:
        lowest_dist = avrg_dist
        best_charger = charging_station

end_time = time.time()

elapsed_time = end_time - start_time
print(f"Þetta tók: {elapsed_time:.4f} sekúndur")
print(f'Besta hleðslustöð er: {best_charger} með meðalvegalengdina: {lowest_dist}')

Fjöldi mögulegra hleðslustöðva: 1829
Hleðslustöð: 12885876, meðalvegalengd (m): 8917.937201746985
Hleðslustöð: 12885924, meðalvegalengd (m): 9486.838209973888
Hleðslustöð: 12885930, meðalvegalengd (m): 8921.25793108706
Hleðslustöð: 12885952, meðalvegalengd (m): 8927.71241230635
Hleðslustöð: 12885979, meðalvegalengd (m): 8956.779660572547
Hleðslustöð: 12885991, meðalvegalengd (m): 8934.149766171342
Hleðslustöð: 12886003, meðalvegalengd (m): 9493.384632278869
Hleðslustöð: 12886027, meðalvegalengd (m): 8942.76500226823
Hleðslustöð: 14581636, meðalvegalengd (m): 11164.98613235218
Hleðslustöð: 14581771, meðalvegalengd (m): 11200.800411071508
Hleðslustöð: 14581824, meðalvegalengd (m): 11118.868892928247
Hleðslustöð: 14581900, meðalvegalengd (m): 11140.040561627728
Hleðslustöð: 14581950, meðalvegalengd (m): 11145.88330627996
Hleðslustöð: 14581956, meðalvegalengd (m): 10490.497688816251
Hleðslustöð: 14581975, meðalvegalengd (m): 11174.632534211245
Hleðslustöð: 14581979, meðalvegalengd (m): 111

Besta hleðslustöð er: 560670782 með meðalvegalengdina: 15633.953016765261 - þessi staðsetning er á miðju höfuðborgarsvæðinu, þar sem Reykjanesbraut og Breiðholtsbraut mætast.

### 7. Gráðug reiknirit (⋆⋆)

Útfærið gráðugt reiknirit sem leitar að bestu lausn fyrir k = 2, . . . , 10 með því að leysa
vandamálið fyrir k−1 hleðslustöðvum og bæta þá við þann hnút sem gefur minnsta markfall,
miðað við að ekki sé hægt að breyta v1, . . . , vk−1.
Sýnið á korti hvaða hleðslustöðvar eru valdar fyrir k = 10 og mælið tímann sem reikniritið
tekur.

In [153]:
start_time = time.time()

best_stations = []
results = None

for k in range(1, 11):  # k frá 1 til 10
    print(f"Finna stöð nr {k}...")
    lowest_avg = float('inf')
    best_candidate = None

    for candidate in possible_charging_stations:
        if candidate in best_stations:
            continue  # sleppum þeim sem eru nú þegar valdar

        trial_stations = best_stations + [candidate]
        trial_results = shortest_paths_to_charging_stations_using_Astar_faster(G, trial_stations, nodes)
        trial_avg = average_distance(trial_results)

        if trial_avg < lowest_avg:
            lowest_avg = trial_avg
            best_candidate = candidate

    best_stations.append(best_candidate)

end_time = time.time()
elapsed_time = end_time - start_time

print("Bestu hleðslustöðvarnar:")
for i, cs in enumerate(best_stations, 1):
    print(f"{i}. {cs}")
print(f"Þetta tók: {elapsed_time:.4f} sekúndur")

# fjöldi mögulega hleðslustöðva: 1829
# Finna stöð nr 1...
# Finna stöð nr 2...
# Finna stöð nr 3...
# Finna stöð nr 4...
# Finna stöð nr 5...
# Finna stöð nr 6...
# Finna stöð nr 7...
# Finna stöð nr 8...
# Finna stöð nr 9...
# Finna stöð nr 10...
# Bestu hleðslustöðvarnar:
# 1. 560670782
# 2. 5541083892
# 3. 10740089234
# 4. 470316420
# 5. 251765349
# 6. 8138139106
# 7. 1420012506
# 8. 111474147
# 9. 2948755310
# 10. 252165202
# Þetta tók: 2360.9228 sekúndur


Finna stöð nr 1...
Finna stöð nr 2...
Finna stöð nr 3...
Finna stöð nr 4...
Finna stöð nr 5...
Finna stöð nr 6...
Finna stöð nr 7...
Finna stöð nr 8...
Finna stöð nr 9...
Finna stöð nr 10...
Bestu hleðslustöðvarnar:
1. 2469557257
2. 3865596367
3. 560670782
4. 10740089234
5. 470316420
6. 35786586
7. 295786685
8. 1420012506
9. 269072778
10. 111474147
Þetta tók: 691.2738 sekúndur


In [154]:
import folium

def plot_charging_stations_on_map(best_stations, nodes):
    """Teiknar hleðslustöðvar á kort"""
    
    # Náum í hnit bestu hleðslustöðvanna
    station_coords = nodes[nodes["osmid"].isin(best_stations)][["x", "y"]]

    # Setjum miðju kortsins á meðaltal hnitanna
    map_center = [station_coords["y"].mean(), station_coords["x"].mean()]
    folium_map = folium.Map(location=map_center, zoom_start=13)

    # Merkjum hleðslustöðvarnar á kortinu
    for _, row in station_coords.iterrows():
        folium.Marker(
            location=[row["y"], row["x"]],
            popup=f"Station ID: {row.name}",
            icon=folium.Icon(color="blue", icon="charging-station", prefix="fa"),
        ).add_to(folium_map)
        
    return folium_map

In [155]:
map = plot_charging_stations_on_map(best_stations, nodes)
map

### 8. Skárri gráðug reiknirit (⋆⋆)

Gráðuga reikniritið á það til að mála sig út í horn með því að velja lélegan fyrsta hnút.
Breytið leitinni þannig að þið veljið handahófskenndan fyrsta hnút og farið endurkvæmt í
tilfellin k = 2, . . . , 10. Í hverju undirtilfelli finnið þið 2 bestu hnútana sem koma til greina
en eru langt frá hvor öðrum og prófið endurkvæmt alla möguleika. Haldið utan um bestu
lausnina sem finnst fyrir nokkur hanndahófskennda upphafspunkta og sýnið bestu lausn á
korti. Hve mikinn tíma tekur reikniritið ykkar?

In [ ]:
def find_two_possible_nodes(G, nodes, best_stations, remaining_nodes):
    """
    Finnur tvo góða hnúta sem eru langt frá hverjum öðrum

    Inntak: 
    - G: netið
    - nodes: DataFrame með öllum hnútnum
    - best_stations: listi af bestu hleðslustöðvunum sem hafa verið valdnar
    - remaining_nodes: dataframe yfir alla hnúta sem er hægt að velja úr

    Skilar:
    - best_candidate1: fyrsti valdi hnúturinn
    - best_candidate2: annar valdi hnúturinn
    """
    lowest_avg = float('inf')
    best_candidate1 = None 
    best_candidate2 = None

    # Finnum fyrst fyrsta hnútinn sem er besti möguleikinn
    for candidate in remaining_nodes["osmid"]:
        trial_stations = best_stations + [candidate]
        trial_results = shortest_paths_to_charging_stations_using_Astar_faster(G, trial_stations, nodes) # nodes þarf að vera dataframe
        trial_avg = average_distance(trial_results)

        if trial_avg < lowest_avg:
            lowest_avg = trial_avg
            best_candidate1 = candidate

    print(f"Fyrsti valdi hnúturinn: {best_candidate1}")
    remaining_nodes = remaining_nodes[~remaining_nodes["osmid"].isin([best_candidate1])] # fjarlægjum valda hnútinn úr remaining_nodes

    # Finnum svo hnút sem er næst-bestur en amk 10.000 metra frá fyrsta hnútnum sem við völdum
    # Remove nodes from remaining_nodes by running dijkstra and picking the nodes that are closest to the best_candidate1
    # The threshold is 10.000 meters
    threshold = 10000
    distances, prev = dijkstra(G, best_candidate1) # kannski breyta seinna þannig reiknum bara frá primary hnútum
    closest_nodes = [node for node, dist in distances.items() if dist < threshold and node not in best_stations]
    furthest_nodes = remaining_nodes[~remaining_nodes["osmid"].isin(closest_nodes)]

    lowest_avg = float('inf')

    for candidate in furthest_nodes["osmid"]:
        trial_stations = best_stations + [candidate]
        trial_results = shortest_paths_to_charging_stations_using_Astar_faster(G, trial_stations, nodes)
        trial_avg = average_distance(trial_results)

        if trial_avg < lowest_avg:
            lowest_avg = trial_avg
            best_candidate2 = candidate

    print(f"Annar valdi hnúturinn: {best_candidate2}")
            
    return best_candidate1, best_candidate2


In [ ]:
import random

def recursive_selection(G, nodes, possible_charging_stations, k=10, best_stations=None, depth=0):
    """Velur handahófskennt fyrsta hnútinn og svo endurkvæmt hina k-1 hnútana. 
       Í hverju undirverkefni eru tveir bestu hnútarnir sem koma til greina en 
       eru langt frá hvor öðrum fundnir og allir möguleikar prófaðir endurkvæmt.
       
       Inntak: 
       G: netið
       nodes: dataframe yfir hnútana
       possible_charging_stations: dataframe yfir mögulegar hleðslustöðvar
       k: fjöldi hnúta sem á að velja
       best_stations: listi af hnútum sem hafa verið valdir sem bestu hleðslustöðvarnar

       setja í priority queue

       hvora lausnina ættum við að velja?
       
       Úttak: 
       listi af völdum hnútum
    """

    # Frumstillum best_stations ef það er tómt
    if best_stations is None:
        best_stations = []

    # Grunntilfelli: stoppum þegar k hnútar hafa verið valdir
    if len(best_stations) == k:
        return best_stations

    # Ef það er ekki búið að velja fyrsta hnútinn, veljum við hann handahófskennt
    if not best_stations:
        first_node = random.choice(possible_charging_stations["osmid"].tolist())
        best_stations.append(first_node)

    print(f"Fyrsta hleðslustöðin (valin handahófskennt): {best_stations[-1]}")
    print(f"Finna tvo möguleika fyrir stöð nr {depth + 1}...")
    # Finnum tvo mögulega hnúta sem eru langt frá hverjum öðrum
    remaining_nodes = possible_charging_stations[~possible_charging_stations["osmid"].isin(best_stations)] # tökum frá hnútana sem hafa verið valdnir
    candidate1, candidate2 = find_two_possible_nodes(G, nodes, best_stations, remaining_nodes) # veljum tvo hnúta sem eru bestir og langt frá hvor öðrum

    if candidate1 is None or candidate2 is None:
        print("Einn eða fleiri kandidatar eru None - hættum þessari grein af trénu.")
        return best_stations

    option1 = recursive_selection(G, nodes, possible_charging_stations, k, best_stations + [candidate1], depth + 1)
    option2 = recursive_selection(G, nodes, possible_charging_stations, k, best_stations + [candidate2], depth + 1)

    option1_avg = average_distance(shortest_paths_to_charging_stations_using_Astar_faster(G, option1, nodes))
    option2_avg = average_distance(shortest_paths_to_charging_stations_using_Astar_faster(G, option2, nodes))

    return option1 if option1_avg < option2_avg else option2

In [28]:
possible_charging_stations_small = small_nodes[small_nodes["primary"]]

two_best_stations = recursive_selection(G_small_dict, small_nodes, possible_charging_stations_small, k=2, best_stations=None)
print("Bestu hleðslustöðvarnar (handahófskennt val):")
for cs in enumerate(two_best_stations, 1):
    print(f"{cs}")

Finna tvo möguleika fyrir stöð nr 1...
Fyrsti valdi hnúturinn: 10799990319
Einn eða fleiri kandidatar eru None - hættum þessari grein af trénu.
Bestu hleðslustöðvarnar (handahófskennt val):
(1, 252744087)


In [32]:
possible_charging_stations = nodes[nodes["primary"]]

best_stations_recursive = recursive_selection(G, nodes, possible_charging_stations, k=10, best_stations=None)
print("Bestu hleðslustöðvarnar (endurkvæmt val):")
for i, cs in enumerate(best_stations_recursive, 1):
    print(f"{i}. {cs}")

Finna tvo möguleika fyrir stöð nr 1...
Fyrsti valdi hnúturinn: 1022635377
Finna tvo möguleika fyrir stöð nr 2...
Fyrsti valdi hnúturinn: 253483797
Finna tvo möguleika fyrir stöð nr 3...
Fyrsti valdi hnúturinn: 35786586
Finna tvo möguleika fyrir stöð nr 4...
Fyrsti valdi hnúturinn: 14772869
Finna tvo möguleika fyrir stöð nr 5...
Fyrsti valdi hnúturinn: 3542177123
Finna tvo möguleika fyrir stöð nr 6...
Fyrsti valdi hnúturinn: 36166172
Finna tvo möguleika fyrir stöð nr 7...
Fyrsti valdi hnúturinn: 269072778
Finna tvo möguleika fyrir stöð nr 8...
Fyrsti valdi hnúturinn: 6382150974
Finna tvo möguleika fyrir stöð nr 9...
Fyrsti valdi hnúturinn: 286323555
Finna tvo möguleika fyrir stöð nr 9...
Fyrsti valdi hnúturinn: 6382150974
Finna tvo möguleika fyrir stöð nr 8...
Fyrsti valdi hnúturinn: 269072778
Finna tvo möguleika fyrir stöð nr 9...
Fyrsti valdi hnúturinn: 286323555
Finna tvo möguleika fyrir stöð nr 9...
Fyrsti valdi hnúturinn: 286323555


KeyboardInterrupt: 

### 9. Nákvæm lausn fyrir k = 10 (⋆ ⋆ ⋆)

Finnið bestu lausn fyrir k = 10 með því að setja vandamálið upp sem heiltölubestunarverkefni (e. integer linear program) og leysa það með því að nota pakka á borð við Gurobi eða
OR-tools. Athugið að verkefnið gæti verið of stórt fyrir þessa pakka. Nýtið ykkur götur í hverfum eru oft teng við primary hnúta í gegnum einn veg, þá er hægt að einfalda netið með því að skipta þessum hverfum út fyrir einn hnút sem tengir sameiginlegan primary hnút. Þessa leggi er hægt að finna með DFS. Sýnið bestu lausnina á korti og mælið tímann sem
heiltölubestunarverkefnið tekur.


In [ ]:
# Þetta er líklegast ekki besta leiðin til þess að gera þetta allavega miðað við magnið á lykjum í þessu
# en þetta var það eina sem ég gat fundið upp á sem leysir þennan hluta af verkefninu
def find_collapsible_neighborhoods(G):
    """
    Þetta fall finnur "hverfi" sem tengjast bara í einn primary hnút til þess að svo sé hægt
    að taka þessi hverfi út og einfalda netið. Þetta fall fer í gegnum hvern primary hnút og finnur
    hvort hann sé tengdur við svona "hverfi" og safnar hverfunum og primary hnútinum sem það tengist
    í lista sem það skilar svo hægt sé að einfalda netið með þeim upplýsingum
    """

    undirected = G.to_undirected() # þetta er svo við lendum í ákveðnum edge cases sem gefa false positives (erfitt að útskýra idk)
    visited = set() # búum til set sem geymir fyrir okkur þá hnúta sem við höfum skoðað áður svo við skilum aldrei tvisvar sama hverfi
    collapsible = [] # þetta er listi af hverfum og þeim eina primary hnút sem þau tengjast og munum við skila þessu

    for node in G.nodes: # ítrum í gegnum alla hnúta
        if not G.nodes[node]["primary"]: # viljum bara primary hnúta svo sleppum þeim sem eru það ekki
            continue

        for neighbor in undirected.neighbors(node): # kíkjum á alla nágranna valda primary hnútsins
            if neighbor in visited or G.nodes[neighbor]["primary"]: # viljum sleppa nágrannanum ef við höfum skoðað hann áður eða ef hann er primary hnútur
                continue

            stack = [neighbor] # búum til hlaða fyrir DFS-ið okkar
            neighborhood = set() # þetta mun vera hverfið sem við erum að fara ítra í gegnum með DFS
            found_other_primary = False # þetta er boolean gildi sem segir ef við höfum fundið primary hnút í hverfinu sem er ekki upprunaleigi primary hnúturinn

            while stack: #DFS
                current = stack.pop() # finnum hvað við erum að skoða í þessari ítrun
                if current in neighborhood: # ef við höfum nú þegar skoðað þennan þá sleppum við honum
                    continue

                neighborhood.add(current) # bætum hnútinum í hverfið

                for nbr in undirected.neighbors(current): # kíkjum á alla nágranna hnútsins sem við erum að skoða (skýrði þetta nbr því neighbor er nú þegar tekið)
                    if nbr == node: # sleppum hnútinum ef þetta er upprunalegi primary hnúturinn
                        continue
                    if G.nodes[nbr]["primary"]: # ef hnúturinn er primary þá höfum við fundið annan hnút í hverfinu sem er primary
                        found_other_primary = True
                    elif nbr not in neighborhood: # ef við höfum ekki skoðað nágrannan þá setjum við hann á hlaðan til að skoða
                        stack.append(nbr)

            if not found_other_primary: # ef við fundum engan annan primary hnút
                collapsible.append((node, neighborhood)) # þá er þetta hverfi sem tengist bara einum primary hnút og þá er þetta eitthvað sem við munum einfalda

            visited.update(neighborhood) # pössum okkur að bæta öllum hnútum sem við vorum að skoða í visited til að muna það

    return collapsible # skilum lista af öllum valid hverfum

In [ ]:
# tilraun tvö sem leifir hverfi sem innihalda primary hnúta
def find_collapsible_neighborhoods2(G):
    """
    Þetta fall finnur "hverfi" sem tengjast bara í einn primary hnút til þess að svo sé hægt
    að taka þessi hverfi út og einfalda netið. Þetta fall fer í gegnum hvern primary hnút og finnur
    hvort hann sé tengdur við svona "hverfi" og safnar hverfunum og primary hnútinum sem það tengist
    í lista sem það skilar svo hægt sé að einfalda netið með þeim upplýsingum
    """

    undirected = G.to_undirected() # þetta er svo við lendum í ákveðnum edge cases sem gefa false positives (erfitt að útskýra idk)
    visited = set() # búum til set sem geymir fyrir okkur þá hnúta sem við höfum skoðað áður svo við skilum aldrei tvisvar sama hverfi
    collapsible = [] # þetta er listi af hverfum og þeim eina primary hnút sem þau tengjast og munum við skila þessu

    for node in G.nodes: # ítrum í gegnum alla hnúta
        if not G.nodes[node]["primary"]: # viljum bara primary hnúta svo sleppum þeim sem eru það ekki
            continue

        for neighbor in undirected.neighbors(node): # kíkjum á alla nágranna valda primary hnútsins
            if neighbor in visited or G.nodes[neighbor]["primary"]: # viljum sleppa nágrannanum ef við höfum skoðað hann áður eða ef hann er primary hnútur
                continue

            stack = [neighbor] # búum til hlaða fyrir DFS-ið okkar
            neighborhood = set() # þetta mun vera hverfið sem við erum að fara ítra í gegnum með DFS

            while stack: #DFS
                current = stack.pop() # finnum hvað við erum að skoða í þessari ítrun
                if current in neighborhood: # ef við höfum nú þegar skoðað þennan þá sleppum við honum
                    continue

                neighborhood.add(current) # bætum hnútinum í hverfið

                for nbr in undirected.neighbors(current): # kíkjum á alla nágranna hnútsins sem við erum að skoða (skýrði þetta nbr því neighbor er nú þegar tekið)
                    if nbr == node: # sleppum hnútinum ef þetta er upprunalegi primary hnúturinn
                        continue
                    elif nbr not in neighborhood: # ef við höfum ekki skoðað nágrannan þá setjum við hann á hlaðan til að skoða
                        stack.append(nbr)
            
            if len(neighborhood) < 0.5 * len(G.nodes): # kíkjum hvort hverfið er stærra en 50% af netinu
                collapsible.append((node, neighborhood)) # ef netið er minna en 50% af netinu þá lak það ekki út í restina af netinu
                visited.update(neighborhood) # sem þýðir að við fundum hverfi sem uppfyllir skilirðin

    return collapsible # skilum lista af öllum valid hverfum

In [ ]:
collapsible = find_collapsible_neighborhoods(G)

print(collapsible)

from collections import Counter

node_counts = Counter()
for primary, neighborhood in collapsible:
    node_counts.update(neighborhood)

duplicates = [node for node, count in node_counts.items() if count > 1]
print("Nodes in multiple neighborhoods:", duplicates)

collapsible = find_collapsible_neighborhoods(G)
all_nodes = [node for _, group in collapsible for node in group]
print(len(all_nodes), len(set(all_nodes)))

In [ ]:
import networkx as nx

def collapse_graph_with_average_distance(G, collapsible):
    """
    Þetta fall tekur net og lista af hverfum og þeirra primary hnút og skilar einfölduðu neti
    þar sem hverfin hafa verið breytt í hnúta sem tengjast primary hnútinum sínum
    """

    collapsed_graph = G.copy() # búum til copy af netinu

    for primary_node, neighborhood in collapsible: # tökum primary hnút og lista af hnútum í hverfinu fyrir hvern hlut í listanum
        total_distance = 0
        # þurfum að fylgjast með x og y gildum því þurfum þau á nýja hnútin því A*-ið okkar notar hnit
        total_x = 0
        total_y = 0
        count = 0

        for node in neighborhood:
            if nx.has_path(collapsed_graph, primary_node, node): # passar að í collapsed netinu er hægt að fara frá primary í hvert node
                distance = nx.shortest_path_length(collapsed_graph, primary_node, node) # bætir við distance miðað við styðstu leið
                total_distance += distance # safnar upp í breytu sem geymir samtals lengd
                total_x += G.nodes[node]["x"]
                total_y += G.nodes[node]["y"]
                count += 1

        if count > 0: # pössum að hverfið sé ekki tómt
            average_distance = total_distance / count
            average_x = total_x / count
            average_y = total_y / count
        else: # þetta ætti aldrei að gerast
            average_distance = 0
            average_x = 0
            average_y = 0

        collapsed_node = next(iter(neighborhood)) # býr til nafn fyrir nýja hnútinn með því að taka það frá einum af hnútunum í hverfinu
        collapsed_graph.remove_nodes_from(neighborhood) # eyðir öllum hverfis hnútunum úr netinu sem við ætlum að skila
        collapsed_graph.add_node(collapsed_node, primary=False, x=average_x, y=average_y) # bætir við nýjum non primary hnút
        collapsed_graph.add_edge(primary_node, collapsed_node, weight=average_distance) # gerir tengingu frá nýja hnútnum til primary hnútsins

    return collapsed_graph # skilar loka netinu þar sem hverfin ættu að hafa verið einfölduð í einn hnút


In [ ]:
# test

G = make_graph(nodes, edges)
print("graph made")
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

collapsible = find_collapsible_neighborhoods(G)
print("found collapsible neighborhoods")

collapsed_G = collapse_graph_with_average_distance(G, collapsible)
print("simplified graph")
print("Number of nodes:", collapsed_G.number_of_nodes())
print("Number of edges:", collapsed_G.number_of_edges())

In [ ]:
# smá breytt A* faster sem núna gefur lengd allra hnúta til allra primary hnúta
def all_shortest_paths_to_primary_nodes_faster(G, primary_nodes):
    """Finnur lengd allra hnúta í netinu til allra primary hnúta í netinu"""

    distances = {} # það sem við munum skila
    reversed_G = G.reverse(copy=True) # snúum netinu við

    for cs in primary_nodes: # förum yfir öll primary hnúta

        heuristic = { # reiknum "heuristic" gildið fyrir hvern hnút miðað við valda primary hnútin
            n: math.sqrt((reversed_G.nodes[n]["x"] - reversed_G.nodes[cs]["x"])**2 +
                         (reversed_G.nodes[n]["y"] - reversed_G.nodes[cs]["y"])**2)
            for n in reversed_G.nodes
        }

        open_set = [] # listi fyrir hlaðan
        g_score = defaultdict(lambda: float('inf')) # g gildið sem byrjar sem inf

        g_score[cs] = 0 # g gildið á stöðinni sjálfri er 0
        heapq.heappush(open_set, (heuristic[cs], cs)) # setjum stöðina sjálfa á hlaðan til þess að byrja á

        while open_set: # ítrum í gegnum hlaðan
            _, current = heapq.heappop(open_set) # fáum efsta hlutinn á hlaðanum

            for neighbor in reversed_G.neighbors(current): # fáum nágranna valda hnútsins
                weight = reversed_G[current][neighbor]["weight"] # finnum lengdina milli valda og nágrannans
                tentative_g = g_score[current] + weight # reiknum mögulega g gildið á nágrannanum

                if tentative_g < g_score[neighbor]: # ef mögulega g gildið er minna en g gildið sem það hefur núna
                    g_score[neighbor] = tentative_g # þá setjum við mögulega sem raun g gildið
                    f_score = tentative_g + heuristic[neighbor] # f gildið er þá nýja g gildið plús h gildið
                    heapq.heappush(open_set, (f_score, neighbor)) # setjum inn nágrannan og f gildi hans

        for node, dist in g_score.items(): # ítrum gegnum g gilda listan og fáum hnútin og lengd frá valda primary hnútnum
            if dist < float('inf'): # ef fannst leið frá upprunalega primary hnútnum
                distances[(node, cs)] = dist # þá bætum við hnúta parinu og lengd þeirra á milli í dist listan

    return distances # skilum lengdar listanum

In [ ]:
pip install ortools

In [ ]:
from ortools.linear_solver import pywraplp

def place_k_stations(G, k):
    """Fall sem notar OR-tools til að velja bestu k stöðvarnar"""
    nodes = list(G.nodes) # fáum lista af öllum hnútum
    
    # búum til uppflettitöflu sem geymir lengdir allra hnúta til allra primary hnúta
    all_distances = all_shortest_paths_to_primary_nodes_faster(G, nodes)

    # störtum solvernum
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver: # pössum að allt virkar
        print("Solver not available.")
        return

    # hérna erum við með tvær breytur y og x
    # y[j] = 1 ef hnútur j er valinn sem stöð
    y = {j: solver.BoolVar(f'y_{j}') for j in nodes}

    # x[i,j] = 1 ef hnútur i er tengdur stöð j (semsagt ef j er talið vera besta stöð i)
    x = {}
    for i in nodes:
        for j in nodes:
            x[(i, j)] = solver.BoolVar(f'x_{i}_{j}')

    # hérna setjum við hvað þarf að vera satt
    # við þurfum alltaf að velja k margar stöðvar (hjá okkur mun það alltaf vera 10)
    solver.Add(solver.Sum([y[j] for j in nodes]) == k)

    # hver hnútur þarf að tengjast nákvæmlega einni stöð
    for i in nodes:
        solver.Add(solver.Sum([x[(i, j)] for j in nodes]) == 1)

    # hnútur má bara vera tengdur primary hnút sem er stöð
    for i in nodes:
        for j in nodes:
            solver.Add(x[(i, j)] <= y[j])

    # Hérna segjum við að við erum að reyna minnka meðal lengd frá stöð fyrir hvern hnút
    objective = solver.Objective()
    for i in nodes:
        for j in nodes:
            dist = all_distances.get((i, j), float('inf'))
            if dist < float('inf'):
                objective.SetCoefficient(x[(i, j)], dist)
            else:
                solver.Add(x[(i, j)] == 0)
    objective.SetMinimization()

    # hérna er hann að leysa
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        selected_stations = [j for j in nodes if y[j].solution_value() == 1]
        assignments = {i: j for i in nodes for j in nodes if x[(i, j)].solution_value() == 1}
        print(f"Selected {k} stations at nodes: {selected_stations}")
        print("Assignments:")
        for i in nodes:
            print(f"  Node {i} → Station {assignments[i]}")
        print("Total distance:", objective.Value())
        return selected_stations, assignments
    else:
        print("No optimal solution found.")
        return None, None


In [ ]:
# test

G = make_graph(nodes, edges)
print("graph made")

start_time = time.time()

# comment-að út því þetta skilar villu eftir langan tíma
# selected, assignments = place_k_stations(collapsed_G, 10)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Reikniritið tók: {elapsed_time:.4f} sekúndur")

# print(selected)